In [1]:
import sqlite3
import pandas as pd

database = 'sqlite_db_pythonsqlite.db'

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def make_selection(database,query):
    """ creates a pandas dataframe out of a selection """
    conn = create_connection(database)
    cur = conn.cursor()
    cur.execute(query)
    
    col_names = list(map(lambda x: x[0], cur.description))
    df = pd.DataFrame(cur.fetchall(),columns=col_names)
    return df


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members1!

In [3]:
query = '''
SELECT facility, revenue
FROM (
    SELECT facility, SUM(total_cost) AS revenue
    FROM (
        SELECT F.name AS facility, 
            (CASE WHEN B.memid = 0 THEN B.slots*guestcost
             ELSE B.slots*membercost
             END) AS total_cost
        FROM Bookings AS B
        INNER JOIN Facilities AS F
        ON B.facid = F.facid ) AS subq_1
    GROUP BY facility ) AS subq_2
WHERE revenue < 1000
ORDER BY revenue DESC
'''

make_selection(database,query)

2.6.0


,facility,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [80]:
query = '''
SELECT m1.surname ||', '|| m1.firstname AS member,
       m2.surname ||', '|| m2.firstname AS recommended_by
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY member
'''

make_selection(database,query)

2.6.0


,member,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


Q12: Find the facilities with their usage by member, but not guests 

In [5]:
query = '''
SELECT F.name AS facility, COUNT(*) AS member_usage
FROM Facilities as F
JOIN Bookings as B
ON F.facid = B.facid
WHERE B.memid != 0
GROUP BY F.name
'''

make_selection(database,query)

2.6.0


,facility,member_usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


Q13: Find the facilities usage by month, but not guests 

In [81]:
query = '''
SELECT facility, 
       SUM(JULY) AS July,
       SUM(AUGUST) AS Aug, 
       SUM(SEPTEMBER) AS Sept
FROM (
    SELECT facility,
    CASE WHEN month = '07' THEN usages 
         ELSE 0 END AS 'JULY',
    CASE WHEN month = '08' THEN usages 
         ELSE 0 END AS 'AUGUST',
    CASE WHEN month = '09' THEN usages 
         ELSE 0 END AS 'SEPTEMBER'
    FROM (
        SELECT F.name AS facility,
                strftime('%m',starttime) AS month,
                COUNT(strftime('%m',starttime)) AS usages
        FROM Facilities AS F
        JOIN Bookings AS B
        ON F.facid = B.facid
        WHERE B.memid != 0 
        GROUP BY month, F.name
        ) as subq_1
    ) as subq_2
GROUP BY facility
'''

make_selection(database,query)

2.6.0


,facility,July,Aug,Sept
0,Badminton Court,51,132,161
1,Massage Room 1,77,153,191
2,Massage Room 2,4,9,14
3,Pool Table,103,272,408
4,Snooker Table,68,154,199
5,Squash Court,23,85,87
6,Table Tennis,48,143,194
7,Tennis Court 1,65,111,132
8,Tennis Court 2,41,109,126
